In [2]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Configuração para garantir gráficos interativos no Jupyter
pio.renderers.default = "notebook_connected"  # Usar o renderizador correto

def preprocess_data(file_path):
    # Carregar o arquivo CSV
    df = pd.read_csv(file_path)

    # Remover valores nulos
    df = df.dropna()

    # Garantir que as colunas necessárias sejam numéricas
    df['price(dol)'] = pd.to_numeric(df['price(dol)'], errors='coerce')  # Erros serão convertidos para NaN
    df['duration(minutes)'] = pd.to_numeric(df['duration(minutes)'], errors='coerce')

    # Remover linhas com NaN após conversão
    df = df.dropna(subset=['price(dol)', 'duration(minutes)'])

    # Normalização da duração
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    df['duration(minutes)'] = scaler.fit_transform(df[['duration(minutes)']])

    return df

def plot_interactive_scatter(df):
    """Gráfico interativo de dispersão com Plotly."""
    fig = px.scatter(
        df,
        x='duration(minutes)',
        y='price(dol)',
        color='start_city',
        title="Dispersão Interativa: Duração vs. Preço",
        labels={'duration(minutes)': 'Duração (min)', 'price(dol)': 'Preço (USD)'},
        hover_data=['end_city_airline']
    )
    fig.show()

def plot_interactive_area(df):
    """Gráfico interativo de área de preços por cidade."""
    fig = px.area(
        df,
        x='start_city',  # Eixo x será as cidades de partida
        y='price(dol)',  # Eixo y será os preços
        color='start_city',  # Cor por cidade
        title="Distribuição de Preços por Cidade de Partida (Gráfico de Área)",
        labels={'start_city': 'Cidade de Partida', 'price(dol)': 'Preço (USD)'},
        line_group='start_city',  # A linha será agrupada por cidade
        height=600  # Altura do gráfico
    )
    fig.update_traces(mode='lines+markers')  # Mostrar as linhas e os marcadores no gráfico
    fig.show()


def plot_interactive_line(df):
    """Gráfico interativo de linha de preços médios por cidade."""
    avg_price = df.groupby('start_city')['price(dol)'].mean().reset_index()
    fig = px.line(
        avg_price,
        x='start_city',
        y='price(dol)',
        title="Preço Médio por Cidade de Partida",
        labels={'start_city': 'Cidade de Partida', 'price(dol)': 'Preço Médio (USD)'},
        markers=True
    )
    fig.show()

def analyze_file(file_path):
    # Pré-processar os dados
    df = preprocess_data(file_path)

    # Exibir análise descritiva
    print("\nAnálise dos Dados:")
    print(df.describe())

    # Gráficos interativos
    plot_interactive_scatter(df)
    plot_interactive_area(df)
    plot_interactive_line(df)

# Caminho para o arquivo CSV (modifique conforme necessário)
file_path = 'flights.csv'

# Executando a análise
analyze_file(file_path)



Análise dos Dados:
       duration(minutes)  price(dol)
count          72.000000   72.000000
mean            0.130024  389.136944
std             0.215463  173.415213
min             0.000000  221.960000
25%             0.021277  278.955000
50%             0.021277  342.170000
75%             0.143617  430.235000
max             1.000000  992.000000


In [4]:
import plotly.graph_objects as go
import pandas as pd

# Coordenadas das cidades
df = pd.DataFrame({
    'start_city': ['Curitiba', 'Rio de Janeiro', 'Porto Alegre', 'Brasília', 'Belo Horizonte', 
                   'Natal', 'Florianópolis', 'São Paulo', 'Salvador'],
    'end_city_airline': ['Rio de Janeiro', 'Porto Alegre', 'Brasília', 'Belo Horizonte', 'Natal', 
                         'Florianópolis', 'São Paulo', 'Salvador', 'Curitiba'],
    'start_lat': [-25.4296, -22.9068, -30.0346, -15.7801, -19.9191, -5.7945, -27.5954, -23.5505, -12.9714],
    'start_lon': [-49.2717, -43.1729, -51.2177, -47.9292, -43.9386, -35.2110, -48.5480, -46.6333, -38.5014],
    'end_lat': [-22.9068, -30.0346, -15.7801, -19.9191, -5.7945, -27.5954, -23.5505, -12.9714, -25.4296],
    'end_lon': [-43.1729, -51.2177, -47.9292, -43.9386, -35.2110, -48.5480, -46.6333, -38.5014, -49.2717]
})

def plot_map_with_lines(df):
    # Criar a figura do mapa
    fig = go.Figure(go.Scattergeo(
        locationmode='ISO-3',  # Usar código ISO-3 para localização
        lat=df['start_lat'].tolist() + df['end_lat'].tolist(),  # Latitudes
        lon=df['start_lon'].tolist() + df['end_lon'].tolist(),  # Longitudes
        mode='markers+lines',  # Mostrar marcadores e linhas
        marker=dict(size=10, color='blue', symbol='circle', opacity=0.7, line=dict(width=1, color='black')),  # Estilo dos marcadores
        line=dict(width=3, color='red', dash='dot'),  # Estilo das linhas
        text=df['start_city'].tolist() + df['end_city_airline'].tolist(),  # Texto para cada cidade
        hoverinfo='text'  # Exibir nome das cidades no hover
    ))

    # Layout do gráfico
    fig.update_layout(
        geo=dict(
            projection_type='mercator',  # Projeção do mapa
            scope='south america',  # Limitar o mapa para a América do Sul
            center=dict(lat=-14.2350, lon=-51.9253),  # Centralizar o mapa no Brasil
            showland=True,  # Mostrar a terra
            landcolor='rgb(240, 240, 240)',  # Cor do solo
            subunitwidth=1,  # Largura das fronteiras
            countrywidth=2,  # Largura do país
            showcoastlines=True,  # Mostrar as linhas costeiras
            coastlinecolor='rgb(102, 102, 102)',  # Cor das linhas costeiras
            showlakes=True,  # Mostrar lagos
            lakecolor='rgb(255, 255, 255)'  # Cor dos lagos
        ),
        title="Linhas entre Cidades Brasileiras",
        showlegend=False,
        hovermode='closest'
    )

    # Exibir o gráfico
    fig.show()

# Chamar a função para exibir o mapa com linhas
plot_map_with_lines(df)
